detectron2 using cocolike structure training mask rcnn for 5000 iterations.

In [1]:
import torch, torchvision
print(torch.__version__, torch.cuda.is_available(), torch.version.cuda)

1.6.0 True 10.1


In [2]:
import os
import copy
import json
import pycocotools
import random 

#import numpy as np 
#import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from pathlib import Path
from collections import defaultdict

assert torch.__version__.startswith("1.6")

import detectron2
import detectron2.data.transforms as T
import detectron2.utils.comm as comm

from detectron2.utils.logger import setup_logger
setup_logger()

from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg

from detectron2.data import MetadataCatalog,DatasetMapper,build_detection_train_loader,build_detection_test_loader
from detectron2.data import detection_utils as utils
from detectron2.data.catalog import DatasetCatalog
from detectron2.data.datasets import register_coco_instances 

from detectron2.evaluation import COCOEvaluator, inference_on_dataset

from detectron2.projects.deeplab import add_deeplab_config, build_lr_scheduler

In [3]:
PATH = os.path.abspath(os.getcwd())

register_coco_instances("my_dataset_train",{},PATH + "/input/train_annotations.json",PATH + "/input/train_v2/") #TRAIN annotations got mixed up -.-
register_coco_instances("my_dataset_val",{},PATH + "/input/test_annotations.json",PATH + "/input/train_v2/")

my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train")
dataset_dicts = DatasetCatalog.get("my_dataset_train")

[11/27 12:48:50 d2.data.datasets.coco]: Loaded 4255 images in COCO format from /application/input/test_annotations.json


In [4]:
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_X_101_32x8d_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
#cfg.TEST.EVAL_PERIOD = 500 ## // TODO useful to set up eval_hook
cfg.DATALOADER.NUM_WORKERS = 4 ## 4 per gpu
cfg.SOLVER.IMS_PER_BATCH = 32 ## So many that it fulfills the gpu vram
cfg.SOLVER.BASE_LR = 0.0025  # pick a good LR
cfg.SOLVER.MAX_ITER = 500
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 512 
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1  # only has one class (ship)

In [5]:
def custom_mapper(dataset_dict):
    # Implement a mapper, similar to the default DatasetMapper, but with your own customizations
    dataset_dict = copy.deepcopy(dataset_dict)  # it will be modified by code below
    image = utils.read_image(dataset_dict["file_name"], format="BGR")
    transform_list = [T.RandomCrop("absolute",(256,256)), 
                      T.RandomFlip(prob=0.5, horizontal=False, vertical=True),
                      T.RandomFlip(prob=0.5, horizontal=True, vertical=False) 
                     ]
    image, transforms = T.apply_transform_gens(transform_list, image)
    dataset_dict["image"] = torch.as_tensor(image.transpose(2, 0, 1).astype("float32"))

    annos = [
        utils.transform_instance_annotations(obj, transforms, image.shape[:2])
        for obj in dataset_dict.pop("annotations")
        if obj.get("iscrowd", 0) == 0
    ]
    instances = utils.annotations_to_instances(annos, image.shape[:2])
    dataset_dict["instances"] = utils.filter_empty_instances(instances)
    return dataset_dict


from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator
class CocoTrainer(DefaultTrainer):
    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            os.makedirs("coco_eval", exist_ok=True)
            output_folder = "coco_eval"
        return COCOEvaluator(dataset_name, cfg, False, output_folder)
    
    @classmethod
    def build_train_loader(cls, cfg):
        return build_detection_train_loader(cfg, mapper=custom_mapper)
    
    @classmethod
    def build_lr_scheduler(cls, cfg, optimizer):
        return build_lr_scheduler(cfg, optimizer)


In [6]:
%load_ext tensorboard
%tensorboard --logdir logs

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = CocoTrainer(cfg) 
trainer.resume_or_load(resume=True) #True takes last checkpoint file which is saved below.
trainer.train() #Trainer will throw out non-annotated pictures. 

[11/27 13:07:28 d2.engine.defaults]: Model:
GeneralizedRCNN(
  (backbone): FPN(
    (fpn_lateral2): Conv2d(256, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelMaxPool()
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res

[11/27 13:07:28 d2.data.datasets.coco]: Loaded 4255 images in COCO format from /application/input/test_annotations.json
[11/27 13:07:28 d2.data.build]: Removed 0 images with no usable annotations. 4255 images left.
[11/27 13:07:28 d2.data.build]: Using training sampler TrainingSampler
[11/27 13:07:28 d2.data.common]: Serializing 4255 elements to byte tensors and concatenating them all ...
[11/27 13:07:28 d2.data.common]: Serialized dataset takes 2.15 MiB
[11/27 13:07:28 d2.checkpoint.c2_model_loading]: Remapping C2 weights ......
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.bias            loaded from res2_0_branch2a_bn_b              of shape (256,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.running_mean    loaded from res2_0_branch2a_bn_rm             of shape (256,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.0.conv1.norm.running_var     loaded from res2_0_branch2a_bn_riv

[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.2.conv2.weight               loaded from res2_2_branch2b_w                 of shape (256, 8, 3, 3)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.2.conv3.norm.bias            loaded from res2_2_branch2c_bn_b              of shape (256,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.2.conv3.norm.running_mean    loaded from res2_2_branch2c_bn_rm             of shape (256,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.2.conv3.norm.running_var     loaded from res2_2_branch2c_bn_riv            of shape (256,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.2.conv3.norm.weight          loaded from res2_2_branch2c_bn_s              of shape (256,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res2.2.conv3.weight               loaded from res2_2_branch2c_w                 of shape (256, 256, 1, 1

[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res3.2.conv2.norm.running_mean    loaded from res3_2_branch2b_bn_rm             of shape (512,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res3.2.conv2.norm.running_var     loaded from res3_2_branch2b_bn_riv            of shape (512,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res3.2.conv2.norm.weight          loaded from res3_2_branch2b_bn_s              of shape (512,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res3.2.conv2.weight               loaded from res3_2_branch2b_w                 of shape (512, 16, 3, 3)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res3.2.conv3.norm.bias            loaded from res3_2_branch2c_bn_b              of shape (512,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res3.2.conv3.norm.running_mean    loaded from res3_2_branch2c_bn_rm             of shape (512,)
[11/27 

[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.1.conv1.norm.weight          loaded from res4_1_branch2a_bn_s              of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.1.conv1.weight               loaded from res4_1_branch2a_w                 of shape (1024, 1024, 1, 1)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.1.conv2.norm.bias            loaded from res4_1_branch2b_bn_b              of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.1.conv2.norm.running_mean    loaded from res4_1_branch2b_bn_rm             of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.1.conv2.norm.running_var     loaded from res4_1_branch2b_bn_riv            of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.1.conv2.norm.weight          loaded from res4_1_branch2b_bn_s              of shape (1024,)

...

[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.7.conv1.norm.bias            loaded from res4_7_branch2a_bn_b              of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.7.conv1.norm.running_mean    loaded from res4_7_branch2a_bn_rm             of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.7.conv1.norm.running_var     loaded from res4_7_branch2a_bn_riv            of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.7.conv1.norm.weight          loaded from res4_7_branch2a_bn_s              of shape (1024,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.7.conv1.weight               loaded from res4_7_branch2a_w                 of shape (1024, 1024, 1, 1)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res4.7.conv2.norm.bias            loaded from res4_7_branch2b_bn_b              of shape (1024,)

[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.0.conv1.norm.running_var     loaded from res5_0_branch2a_bn_riv            of shape (2048,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.0.conv1.norm.weight          loaded from res5_0_branch2a_bn_s              of shape (2048,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.0.conv1.weight               loaded from res5_0_branch2a_w                 of shape (2048, 1024, 1, 1)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.0.conv2.norm.bias            loaded from res5_0_branch2b_bn_b              of shape (2048,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.0.conv2.norm.running_mean    loaded from res5_0_branch2b_bn_rm             of shape (2048,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.0.conv2.norm.running_var     loaded from res5_0_branch2b_bn_riv            of shape (2048,)

[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.res5.2.conv3.weight               loaded from res5_2_branch2c_w                 of shape (2048, 2048, 1, 1)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.stem.conv1.norm.bias              loaded from res_conv1_bn_b                    of shape (64,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.stem.conv1.norm.running_mean      loaded from res_conv1_bn_rm                   of shape (64,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.stem.conv1.norm.running_var       loaded from res_conv1_bn_riv                  of shape (64,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.stem.conv1.norm.weight            loaded from res_conv1_bn_s                    of shape (64,)
[11/27 13:07:29 d2.checkpoint.c2_model_loading]: backbone.bottom_up.stem.conv1.weight                 loaded from conv1_w                           of shape (64, 3, 7, 7)
[

/opt/conda/lib/python3.7/site-packages/detectron2/structures/masks.py:345: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/opt/conda/lib/python3.7/site-packages/detectron2/structures/masks.py:345: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /opt/conda/conda-bld/pytorch_1595629403081/work/torch/csrc/utils/python_arg_parser.cpp:766.)
  item = item.nonzero().squeeze(1).cpu().numpy().tolist()
/opt/conda/lib/python3.7/site-packages/detectron2/structures/masks.py:345: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzer

[11/27 13:08:01 d2.utils.events]:  eta: 0:12:24  iter: 19  total_loss: 1.677  loss_cls: 0.2651  loss_box_reg: 0.007044  loss_mask: 0.6937  loss_rpn_cls: 0.6911  loss_rpn_loc: 0.02589  time: 1.5538  data_time: 0.2049  lr: 4.9952e-05  max_mem: 7807M
[11/27 13:08:35 d2.utils.events]:  eta: 0:12:12  iter: 39  total_loss: 1.403  loss_cls: 0.04759  loss_box_reg: 0.008121  loss_mask: 0.6909  loss_rpn_cls: 0.6177  loss_rpn_loc: 0.0226  time: 1.6230  data_time: 0.1789  lr: 9.9902e-05  max_mem: 7807M


In [ ]:
## save the training.
checkpointer = DetectionCheckpointer(trainer.model, save_dir="./")
checkpointer.save("model_mask_resnet101_rcnn")  # save to save_dir

In [9]:
#Add shapely! 

# !pip install shapely

     |████████████████████████████████| 1.0 MB 217 kB/s eta 0:00:01
